<a href="https://colab.research.google.com/github/SibongileT/SpotifyYearEndPlaylistEDA/blob/main/GetSongLyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install lyricsgenius
import lyricsgenius
genius = lyricsgenius.Genius("2rxZwYupzW9ZPWxmSBe0tJZB15_-kuco7H1DepqINhhdjrACt4_RPRDF0OrQDdiv")
genius.remove_section_headers = True

In [18]:
path = "/content/drive/MyDrive/SpotifyEDA/playlist_data.csv"
playlist_df = pd.read_csv(path,index_col=0)

In [4]:
playlist_df.head()

,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,year,playlist_title
0,Sex With Me,Rihanna,0.580,0.532,1,-5.668,0,0.0424,0.2940,0.000000,0.0948,0.390,112.049,206400,4,20.87658,10,2016,Your top songs 2016
1,Dope,Fifth Harmony,0.719,0.519,0,-6.545,1,0.0494,0.1300,0.000000,0.0833,0.122,90.995,212626,4,32.72684,13,2016,Your top songs 2016
2,Cranes in the Sky,Solange,0.719,0.615,6,-5.972,1,0.0450,0.4080,0.000147,0.2350,0.465,96.988,250973,4,18.63714,5,2016,Your top songs 2016
3,For Free (feat. Drake),DJ Khaled,0.916,0.571,10,-8.241,0,0.2000,0.0173,0.000001,0.1530,0.498,103.521,183695,4,29.67050,10,2016,Your top songs 2016
4,Crush,Yuna,0.678,0.414,9,-10.144,0,0.0498,0.3660,0.000004,0.0885,0.355,130.031,243133,4,29.11098,9,2016,Your top songs 2016


## Get Song Lyrics From Genius
* Clean up artist and song titles so that it matches what is in genius

In [19]:
primary_artist = []
for artist in playlist_df.artist:
    if ',' in artist:
        primary_artist.append(artist.split(',')[0].replace('[', '').replace(']', '').replace("'","").replace('"',""))
    else:
        primary_artist.append(artist.replace('[', '').replace(']', '').replace("'","").replace('"',""))

In [20]:
song_titles = []
import re
import numpy as np

for song in playlist_df.name:
    song = re.split(', |\(|-|!', song)
    song_str = [x.strip(' ') for x in song]
    if 'Remix' in song_str:
        song = song[0] + ' Remix'
        song_titles.append(song)
    else:
        song_titles.append(song[0])

In [21]:
playlist_df['songs'] = song_titles

In [22]:
playlist_df['primary_artist'] = primary_artist

In [23]:
playlist_df.head()

,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,year,playlist_title,id,songs,primary_artist
0,Sex With Me,Rihanna,0.580,0.532,1,-5.668,0,0.0424,0.2940,0.000000,0.0948,0.390,112.049,206400,4,20.87658,10,2016,Your top songs 2016,6WAxKNbZoGNzQTIvdMqkFs,Sex With Me,Rihanna
1,Dope,Fifth Harmony,0.719,0.519,0,-6.545,1,0.0494,0.1300,0.000000,0.0833,0.122,90.995,212626,4,32.72684,13,2016,Your top songs 2016,660JUD1xQLB4XR1dMxUK4x,Dope,Fifth Harmony
2,Cranes in the Sky,Solange,0.719,0.615,6,-5.972,1,0.0450,0.4080,0.000147,0.2350,0.465,96.988,250973,4,18.63714,5,2016,Your top songs 2016,48EjSdYh8wz2gBxxqzrsLe,Cranes in the Sky,Solange
3,For Free (feat. Drake),DJ Khaled,0.916,0.571,10,-8.241,0,0.2000,0.0173,0.000001,0.1530,0.498,103.521,183695,4,29.67050,10,2016,Your top songs 2016,4JDZl9nKIAhAhDjw753u4X,For Free,DJ Khaled
4,Crush,Yuna,0.678,0.414,9,-10.144,0,0.0498,0.3660,0.000004,0.0885,0.355,130.031,243133,4,29.11098,9,2016,Your top songs 2016,3Txcx4jhuiTZSvhAL0WaRc,Crush,Yuna


In [24]:
count=0
lyrics_df = pd.DataFrame(columns=['artist','song','lyrics','song_id'])
artists = []
songs= []
song_ids = []
lyrics = []
added_to_list_count = 0
for ix,artist in enumerate(playlist_df.primary_artist):
    print(count)
    count+=1
    song_title = playlist_df.iloc[ix]['songs']
    try:
        song = genius.search_song(song_title, artist)
        try:
            print("Song with lyrics added to list\n")
            lyrics.append(song.lyrics)
            artists.append(artist)
            songs.append(song_title)
            song_ids.append(playlist_df.iloc[ix]['id'])
        except:
            print("Song without lyrics added to list\n")
            lyrics.append(None)
            artists.append(artist)
            songs.append(song_title)
            song_ids.append(playlist_df.iloc[ix]['id'])
    except:
        pass
    if count%250 == 0:
            print(ix)
            temp_df = pd.DataFrame(columns=['artist','song','lyrics','song_id'])
            temp_df['artist'] = artists
            temp_df['song'] = songs
            temp_df['lyrics'] = lyrics
            temp_df['song_id'] = song_ids
            artists = []
            songs= []
            song_ids = []
            lyrics = []
            lyrics_df = lyrics_df.append(temp_df,ignore_index = True)
            lyrics_df.to_pickle("lyrics3.pkl")

0
Searching for "Sex With Me" by Rihanna...
Done.
Song with lyrics added to list

1
Searching for "Dope" by Fifth Harmony...
Done.
Song with lyrics added to list

2
Searching for "Cranes in the Sky" by Solange...
Done.
Song with lyrics added to list

3
Searching for "For Free " by DJ Khaled...
Done.
Song with lyrics added to list

4
Searching for "Crush" by Yuna...
Done.
Song with lyrics added to list

5
Searching for "Needed Me" by Rihanna...
Done.
Song with lyrics added to list

6
Searching for "Once Upon a Time" by Mariahlynn...
Done.
Song with lyrics added to list

7
Searching for "Ultralight Beam" by Kanye West...
Done.
Song with lyrics added to list

8
Searching for "Scales " by Solange...
Done.
Song with lyrics added to list

9
Searching for "Closer" by The Chainsmokers...
Done.
Song with lyrics added to list

10
Searching for "Dangerous Woman" by Ariana Grande...
Done.
Song with lyrics added to list

11
Searching for "Wonderful" by Lianne La Havas...
Done.
Song with lyrics adde

In [12]:
song_title = playlist_df.iloc[0]['songs']

In [14]:
playlist_df.iloc[0]['primary_artist']

'Rihanna'

In [15]:
song = genius.search_song('Sex With Me', 'Rihanna')

Searching for "Sex With Me" by Rihanna...
Done.


In [16]:
song.lyrics

"Sex with me, so amazing\nAll this all hard work, no vacation\nStay up off my Instagram, pure temptation\nHit a switch on a fake nigga, like a station\nSex with me, so amazing\nSex with me, so amazing\n\nVodka and water, and a lemon\nAnd a few other things I cannot mention\nOh-na-na-na-na\nFive fingers on it (five fingers)\nHit it like you own it, I’ma hit it like I’m on it\nStraight shots of the blue dot\n(Shots, shots, shots, shots)\nBaby, I’ma pick your  poison\nOh-we, ah-yeah, you gon’ need it\nI’m off that la-la\nI’ma get it wet like jacuzzi, ah-yea\n\nAnd sex with me, so amazing\nAll this all hard work, no vacation\nStay up off my Instagram, pure temptation\nSex with me, sex with me, sex with me\nSo amazing, so amazing, mmmm\n\nYou know I got the sauce (sauce), you know I’m saucy\nAnd it’s always wet, a bitch never ever had to use lip gloss on it\nI’ma need you deeper than six, it's not a coffin\nWe’re not making love, tryna get nasty\nWrap up your drugs, come make me happy\nSex 

In [25]:
lyrics_df

,artist,song,lyrics,song_id
0,Rihanna,Sex With Me,"Sex with me, so amazing\nAll this all hard wor...",6WAxKNbZoGNzQTIvdMqkFs
1,Fifth Harmony,Dope,"New Jersey tongue, not saying too much\nI'm fe...",660JUD1xQLB4XR1dMxUK4x
2,Solange,Cranes in the Sky,I tried to drink it away\nI tried to put one i...,48EjSdYh8wz2gBxxqzrsLe
3,DJ Khaled,For Free,Would you fuck me for free?\nAnother one (Anot...,4JDZl9nKIAhAhDjw753u4X
4,Yuna,Crush,You call me on a lazy afternoon\nAsking me wha...,3Txcx4jhuiTZSvhAL0WaRc
...,...,...,...,...
495,Solange,Binz,"...bae...\n\nAh, ah-ah, ah-ah, ah-ah\nAh, ah-a...",7yvdp8dqmxExSJwT2fn6Xq
496,Moses Sumney,Cut Me,"When I'm weary and so worn out\nOoh, when my m...",6YheRoPUnQc0mXiWBUiQz5
497,Snoh Aalegra,Situationship,"So many times that I tried to get over you, no...",1WYjNYaGu2nH0tcb5h2PKi
498,Moneybagg Yo,Said Sum Remix,"Yeah (Turn me up, YC)\nJT\n\nUh, fuck nigga, I...",7HbnX1fxCAoPa7nRVFSEFW


In [26]:
lyrics_df.to_csv('lyrics_data.csv')
!cp lyrics_data.csv "drive/My Drive/SpotifyEDA"